# Q1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [2]:
#Importing Libraries and Packages
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import requests
import re
import time
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

In [7]:
#Activating the chrome driver
driver = webdriver.Chrome(r'G:/DataTrained n FlipRobo_Pojects/chromedriver.exe')

In [3]:
#Opening the homepge of amazon.in
driver.get('https://www.amazon.in/')
#asking the user to input the keywords to search
user_inp = input('Enter the product you want to search:')

Enter the product you want to search:mouse


In [4]:
search_bar = driver.find_element_by_id("twotabsearchtextbox")    # Locating searc_bar by id
search_bar.clear()                                               # clearing search_bar
search_bar.send_keys(user_inp)                                   # sending user input to search bar
search_button = driver.find_element_by_xpath('//div[@class="nav-search-submit nav-sprite"]/span/input')       # Locating search_button by xpath
search_button.click()                                                                # Clicking the button to start search

# 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings","Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [5]:
start_page = 0
end_page = 3
urls = []
for page in range(start_page,end_page+1):
    try:
        page_urls = driver.find_elements_by_xpath('//a[@class="a-link-normal s-no-outline"]')
        
        # appending all the urls on current page to urls list
        for url in page_urls:
            url = url.get_attribute('href')     # Scraping the url from webelement
            if url[0:4]=='http':                # Checking if the scraped data is a valid url or not
                urls.append(url)                # Appending the url to urls list
        print("Product urls of page {} has been scraped.".format(page+1))
        
        # Moving to next page
        nxt_button = driver.find_element_by_xpath('//li[@class="a-last"]/a')      # Locating the next_button which is active
        if nxt_button.text == 'Next→':                                            # Checking if the button located is next button
            nxt_button.click()                                                    # Clicking the next button
            time.sleep(5)                                                         # time delay of 5 seconds
        # If the current active button is not next button, the we will check if the next button is inactive or not    
        elif driver.find_element_by_xpath('//li[@class="a-disabled a-last"]/a').text == 'Next→':    
            print("No new pages exist. Breaking the loop")  # Printing message and breakinf loop if we have reached the last page
            break
            
    except StaleElementReferenceException as e:             # Handling StaleElement Exception   
        print("Stale Exception")
        next_page = nxt_button.get_attribute('href')        # Extracting the url of next page
        driver.get(next_page)                               # ReLoading the next page

Product urls of page 1 has been scraped.
Product urls of page 2 has been scraped.
Product urls of page 3 has been scraped.
Product urls of page 4 has been scraped.


In [6]:
prod_dict = {}
prod_dict['Brand']=[]
prod_dict['Name']=[]
prod_dict['Rating']=[]
prod_dict['No. of ratings']=[]
prod_dict['Price']=[]
prod_dict['Return/Exchange']=[]
prod_dict['Expected Delivery']=[] 
prod_dict['Availability']=[]
prod_dict['Other Details']=[]
prod_dict['URL']=[]

In [7]:
for url in urls[:4]:
    driver.get(url)                                                        # Loading the webpage by url
    print("Scraping URL = ", url)
    #time.sleep(2)
    
    try:
        brand = driver.find_element_by_xpath('//a[@id="bylineInfo"]')      # Extracting Brand from xpath
        prod_dict['Brand'].append(brand.text)
    except NoSuchElementException:
        prod_dict['Brand'].append('-')
    
    try:
        name = driver.find_element_by_xpath('//h1[@id="title"]/span')      # Extracting Name from xpath
        prod_dict['Name'].append(name.text)
    except NoSuchElementException:
        prod_dict['Name'].append('-')
    
    try:
        rating = driver.find_element_by_xpath('//span[@id="acrPopover"]')  # Extracting Ratings from xpath
        prod_dict['Rating'].append(rating.get_attribute("title"))
    except NoSuchElementException:
        prod_dict['Rating'].append('-')
    
    try:
        n_rating = driver.find_element_by_xpath('//a[@id="acrCustomerReviewLink"]/span')     # Extracting no. of Ratings from xpath
        prod_dict['No. of ratings'].append(n_rating.text)
    except NoSuchElementException:
        prod_dict['No. of ratings'].append('-')
        
    try:
        price = driver.find_element_by_xpath('//span[@id="priceblock_ourprice"]')            # Extracting Price from xpath
        prod_dict['Price'].append(price.text)
    except NoSuchElementException:
        prod_dict['Price'].append('-')
    try:                                                                                     # Extracting Return/Exchange policy from xpath
        ret = driver.find_element_by_xpath('//div[@data-name="RETURNS_POLICY"]/span/div[2]/a')
        prod_dict['Return/Exchange'].append(ret.text)
    except NoSuchElementException:
        prod_dict['Return/Exchange'].append('-')
    try:
        delivry = driver.find_element_by_xpath('//div[@id="ddmDeliveryMessage"]/b')         # Extracting Expected Delivery from xpath
        prod_dict['Expected Delivery'].append(delivry.text)
    except NoSuchElementException:
        prod_dict['Expected Delivery'].append('-')
    
    try:
        avl = driver.find_element_by_xpath('//div[@id="availability"]/span')                # Extracting Availability from xpath
        prod_dict['Availability'].append(avl.text)
    except NoSuchElementException:
        prod_dict['Availability'].append('-')
    
    try:                                                                                    # Extracting Other Details from xpath
        dtls = driver.find_element_by_xpath('//ul[@class="a-unordered-list a-vertical a-spacing-mini"]')
        prod_dict['Other Details'].append('  ||  '.join(dtls.text.split('\n')))
    except NoSuchElementException:
        prod_dict['Other Details'].append('-')
    
    prod_dict['URL'].append(url)                                                            # Saving url
    time.sleep(2)

Scraping URL =  https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A0276471YXUIR36CJE2Y&url=%2Felevn-Premium-Wireless-Optical-Desktop%2Fdp%2FB08QWBJ81P%2Fref%3Dsr_1_1_sspa%3Fdchild%3D1%26keywords%3Dmouse%26qid%3D1625243232%26sr%3D8-1-spons%26psc%3D1&qualifier=1625243232&id=8603505263749702&widgetName=sp_atf
Scraping URL =  https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A01661372TW2Q3FU1TDBF&url=%2FPortronics-Wireless-Ergonomic-Receiver-Notebook%2Fdp%2FB08ZJKGMGQ%2Fref%3Dsr_1_2_sspa%3Fdchild%3D1%26keywords%3Dmouse%26qid%3D1625243232%26sr%3D8-2-spons%26psc%3D1&qualifier=1625243232&id=8603505263749702&widgetName=sp_atf
Scraping URL =  https://www.amazon.in/Lenovo-GX30M39704-300-USB-Mouse/dp/B073BRXPZX/ref=sr_1_3?dchild=1&keywords=mouse&qid=1625243232&sr=8-3
Scraping URL =  https://www.amazon.in/Dell-MS116-1000DPI-Wired-Optical/dp/B01HJI0FS2/ref=sr_1_4?dchild=1&keywords=mouse&qid=1625243232&sr=8

In [8]:
prod_df = pd.DataFrame.from_dict(prod_dict)
prod_df

,Brand,Name,Rating,No. of ratings,Price,Return/Exchange,Expected Delivery,Availability,Other Details,URL
0,Visit the elevn Store,elevn ergo11 2.4GHz Premium Wireless Optical M...,4.3 out of 5 stars,233 ratings,₹ 649.00,7 Days Replacement,July 8 - 9,In stock.,"1 YEAR MANUFACTURER'S WARRANTY: For Warranty, ...",https://www.amazon.in/gp/slredirect/picassoRed...
1,Visit the Portronics Store,Portronics Toad 12 Wireless 2.4G Optical Mouse...,4.1 out of 5 stars,"6,165 ratings",₹ 349.00,7 Days Replacement,"Tuesday, July 6",In stock.,[HIGH-FLOW DESIGN] : The overall shape is full...,https://www.amazon.in/gp/slredirect/picassoRed...
2,Visit the Lenovo Store,"Lenovo 300 Wired Plug & Play USB Mouse, High R...",4.4 out of 5 stars,"18,172 ratings",₹ 299.00,7 Days Replacement,"Tuesday, July 6",In stock.,"A high-resolution, 1600 dpi optical sensor ens...",https://www.amazon.in/Lenovo-GX30M39704-300-US...
3,Visit the Dell Store,Dell MS116 1000DPI USB Wired Optical Mouse,4.4 out of 5 stars,"10,228 ratings",₹ 299.00,7 Days Replacement,"Tuesday, July 6",In stock.,"Movement Detection: Optical, Resolution : 1000...",https://www.amazon.in/Dell-MS116-1000DPI-Wired...


In [9]:
#saving data to csv
prod_df.to_csv('Amazon.csv'.format(user_inp))

# 3. Write a python program to access the search bar and search button on images.google.com andscrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [3]:
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import ElementNotInteractableException,StaleElementReferenceException

In [4]:
url = 'https://images.google.com/' #getting the required url
driver.get(url)

search_job = driver.find_element_by_xpath("//input[@class='gLFyf gsfi']") #search bar
search_job.send_keys("fruits") 

search_btn = driver.find_element_by_xpath("//button[@class='Tg7LZd']")
search_btn.click()

In [10]:
#Opening the homepage of images.google.com
driver.get('https://images.google.com/')

In [11]:
# Locating searc_bar by xpath
search=driver.find_element_by_xpath("//input[@class='gLFyf gsfi']")
# Asking the user to input the keywords he/she wants to search
product = input("Enter product name: ")
 # clearing search_bar
search.clear()
# sending user input to search bar
search.send_keys(product)
# Locating search_button by xpath and Clicking the button to start search
search_btn=driver.find_element_by_xpath("//span[@class='z1asCe MZy1Rb']").click()

Enter product name: cars


In [12]:
# 500 time we scroll down by 10000 in order to generate more images on the website
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")

In [13]:
# Scrapping all images URL of specific product
product1=[]
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
    product1.append(i.get_attribute('src'))

In [ ]:
from selenium.webdriver.common.keys import Keys
# Locating searc_bar by xpath and salact all 
driver.find_element_by_xpath("//input[@class='og3lId']").send_keys(Keys.CONTROL+'a')
# Locating searc_bar by xpath and clear all
driver.find_element_by_xpath("//input[@class='og3lId']").send_keys(Keys.DELETE)
# Asking the user to input the keywords he/she wants to search
product = input("Enter product name: ")
# sending user input to search bar
driver.find_element_by_xpath("//input[@class='og3lId']").send_keys(product)
# Locating search_button by xpath and Clicking the button to start search
driver.find_element_by_xpath("//span[@class='n6h3Rc']").click()

In [ ]:
from selenium.webdriver.common.keys import Keys
# Locating searc_bar by xpath and salact all 
driver.find_element_by_xpath("//input[@class='og3lId']").send_keys(Keys.CONTROL+'a')
# Locating searc_bar by xpath and clear all
driver.find_element_by_xpath("//input[@class='og3lId']").send_keys(Keys.DELETE)
# Asking the user to input the keywords he/she wants to search
product = input("Enter product name: ")
# sending user input to search bar
driver.find_element_by_xpath("//input[@class='og3lId']").send_keys(product)
# Locating search_button by xpath and Clicking the button to start search
driver.find_element_by_xpath("//span[@class='n6h3Rc']").click()

In [16]:
# Scrapping all images URL of specific product
product2=[]
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
    product2.append(i.get_attribute('src'))

In [ ]:
# Locating searc_bar by xpath and salact all
driver.find_element_by_xpath("//input[@class='og3lId']").send_keys(Keys.CONTROL+'a')
# Locating searc_bar by xpath and clear all
driver.find_element_by_xpath("//input[@class='og3lId']").send_keys(Keys.DELETE)
# Asking the user to input the keywords he/she wants to search
product = input("Enter product name: ")
# sending user input to search bar
driver.find_element_by_xpath("//input[@class='og3lId']").send_keys(product)
# Locating search_button by xpath and Clicking the button to start search
driver.find_element_by_xpath("//span[@class='n6h3Rc']").click()

In [18]:
# 500 time we scroll down by 10000 in order to generate more images on the website
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")

In [19]:
# Scrapping all images URL of specific product
product3=[]
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
    product3.append(i.get_attribute('src'))

In [20]:
# Make dataframe
Image=pd.DataFrame({})
Image['product1']=product1[0:30]
Image['product2']=product2[0:30]
Image['product3']=product3[0:30]
Image

,product1,product2,product3
0,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
1,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
2,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
3,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
4,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
5,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
6,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
7,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
8,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
9,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."


# Q4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [14]:
def mobileInfo(phone_name,url):
    driver.get(url)

    #Closing the login prompt
    try:
        driver.find_element_by_xpath('//button[@class="_2KpZ6l _2doB4z"]').click()
    except NoSuchElementException:
        pass

    #Typing phone name and clicking search button
    driver.find_element_by_xpath('//div[@class="_3OO5Xc"]/input').send_keys(phone_name)
    driver.find_element_by_xpath('//button[@class="L0Z3Pu"]').click()

    #Initializing list for all the information to be extracted
    url=[]
    brand=[]
    name=[]
    colour=[]
    ram=[]
    storage=[]
    pcamera=[]
    scamera=[]
    dispsize=[]
    dispreso=[]
    processor=[]
    processor_cores=[]
    batt_capacity=[]
    price=[]

    #Extracting URL of all the phones
    delay=10
    WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CLASS_NAME,'_1fQZEK')))
    for i in driver.find_elements_by_xpath('//a[@class="_1fQZEK"]'):
        url.append(i.get_attribute('href'))

    #Iterating through the URL list and extracting all phone's info
    for i in url:
        driver.get(i)
        time.sleep(2)

        #Flag variables
        n=0
        c=0
        r=0
        s=0
        pca=0
        sc=0
        ds=0
        dr=0
        pt=0
        pc=0
        bc=0
        
        #Extracting phone Brand
        brand.append(driver.find_element_by_xpath('//h1[@class="yhB1nd"]/span').text.split()[0])

        #Extracting phone Price
        price.append(driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]').text)

        #Clicking on the readmore label
        driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]').click()
        time.sleep(2)

        #Extracting all the specification's table rows
        specs=driver.find_elements_by_xpath('//tr[@class="_1s_Smc row"]')

        for row in specs:
            label=row.find_element_by_xpath('td[1]').text

            #Extracting phone Name
            if label=='Model Name':
                name.append(row.find_element_by_xpath('td[2]').text)
                n=1

            #Extracting phone Color
            if label=='Color':
                colour.append(row.find_element_by_xpath('td[2]').text)
                c=1

            ##Extracting phone RAM
            if label=='RAM':
                ram.append(row.find_element_by_xpath('td[2]').text)
                r=1

            #Extracting phone Storage
            if label=='Internal Storage':
                storage.append(row.find_element_by_xpath('td[2]').text)
                s=1

            #Extracting phone Primary Camera
            if label=='Primary Camera':
                pcamera.append(row.find_element_by_xpath('td[2]').text)
                pca=1

            #Extracting phone Secondary Camera
            if label=='Secondary Camera':
                scamera.append(row.find_element_by_xpath('td[2]').text)
                sc=1

            #Extracting phone Display Size
            if label=='Display Size':
                dispsize.append(row.find_element_by_xpath('td[2]').text)
                ds=1

            #Extracting phone Resolution
            if label=='Resolution':
                dispreso.append(row.find_element_by_xpath('td[2]').text)
                dr=1

            #Extracting phone Processor
            if label=='Processor Type':
                processor.append(row.find_element_by_xpath('td[2]').text)
                pt=1

            #Extracting phone Processor Core
            if label=='Processor Core':
                processor_cores.append(row.find_element_by_xpath('td[2]').text)
                pc=1

            #Extracting phone Battery Capacity
            if label=='Battery Capacity':
                batt_capacity.append(row.find_element_by_xpath('td[2]').text)
                bc=1

        #Checking Flag variables
        if n==0:
            name.append('-')
        if c==0:
            colour.append('-')
        if r==0:
            ram.append('-')
        if s==0:
            storage.append('-')
        if pca==0:
            pcamera.append('-')
        if sc==0:
            scamera.append('-')
        if ds==0:
            dispsize.append('-')
        if dr==0:
            dispreso.append('-')
        if pt==0:
            processor.append('-')
        if pc==0:
            processor_cores.append('-')
        if bc==0:
            batt_capacity.append('-')

    #Creating DataFrame
    data=pd.DataFrame()
    data['Name']=name
    data['Brand']=brand
    data['Colour']=colour
    data['Ram']=ram
    data['Storage']=storage
    data['Primary Camera']=pcamera
    data['Secondary Camera']=scamera
    data['Display Size']=dispsize
    data['Display Resolution']=dispreso
    data['Processor']=processor
    data['Processor Cores']=processor_cores
    data['Battery Capacity']=batt_capacity
    data['Price']=price
    data['URL']=url
    return data

In [15]:
import pandas as pd 
pd.set_option('display.max_rows',None)
mobile=input('Which phone you want the information of?')
df=mobileInfo(mobile,'https://www.flipkart.com/')
df

Which phone you want the information of?redmi note 10 pro


,Name,Brand,Colour,Ram,Storage,Primary Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor Cores,Battery Capacity,Price,URL
0,Note 10 Pro,REDMI,Vintage Bronze,6 GB,64 GB,64MP + 8MP + 5MP + 2MP,16MP Front Camera,16.94 cm (6.67 inch),2400 x 1080 Pixels,Qualcomm Snapdragon 732G,Octa Core,5020 mAh,"₹17,944",https://www.flipkart.com/redmi-note-10-pro-vin...
1,Note 10 Pro Max,REDMI,Dark Night,6 GB,128 GB,108MP + 8MP + 5MP + 2MP,16MP Front Camera,16.94 cm (6.67 inch),2400 x 1080 Pixels,Qualcomm Snapdragon 732G,Octa Core,5020 mAh,"₹20,896",https://www.flipkart.com/redmi-note-10-pro-max...
2,Note 10 Pro Max,REDMI,Vintage Bronze,6 GB,128 GB,108MP + 8MP + 5MP + 2MP,16MP Front Camera,16.94 cm (6.67 inch),2400 x 1080 Pixels,Qualcomm Snapdragon 732G,Octa Core,5020 mAh,"₹20,788",https://www.flipkart.com/redmi-note-10-pro-max...
3,Note 10 Pro,REDMI,Dark Night,6 GB,128 GB,64MP Rear Camera,-,16.94 cm (6.67 inch),1080 x 2400$$pixel,-,Octa Core,5020 mAh,"₹19,194",https://www.flipkart.com/redmi-note-10-pro-dar...
4,Note 10 Pro,REDMI,Vintage Bronze,6 GB,128 GB,64MP + 8MP + 5MP + 2MP,16MP Front Camera,16.94 cm (6.67 inch),2400 x 1080 Pixels,Qualcomm Snapdragon 732G,Octa Core,5020 mAh,"₹19,229",https://www.flipkart.com/redmi-note-10-pro-vin...
5,Note 10 Pro,REDMI,Vintage Bronze,8 GB,128 GB,64MP Rear Camera,-,16.94 cm (6.67 inch),1080 x 2400$$pixel,-,Octa Core,5020 mAh,"₹21,999",https://www.flipkart.com/redmi-note-10-pro-vin...
6,Note 10 Pro Max,REDMI,Glacial Blue,6 GB,128 GB,108MP + 8MP + 5MP + 2MP,16MP Front Camera,16.94 cm (6.67 inch),2400 x 1080 Pixels,Qualcomm Snapdragon 732G,Octa Core,5020 mAh,"₹20,789",https://www.flipkart.com/redmi-note-10-pro-max...
7,Note 10 Pro,REDMI,Vintage Bronze,6 GB,128 GB,64MP Rear Camera,-,16.94 cm (6.67 inch),1080 x 2400$$pixel,-,Octa Core,5020 mAh,"₹19,165",https://www.flipkart.com/redmi-note-10-pro-vin...
8,Note 10 Pro,REDMI,Glacial Blue,6 GB,128 GB,64MP Rear Camera,-,16.94 cm (6.67 inch),1080 x 2400$$pixel,-,Octa Core,5020 mAh,"₹19,896",https://www.flipkart.com/redmi-note-10-pro-gla...
9,Note 10 Pro Max,REDMI,Vintage Bronze,8 GB,128 GB,108MP + 8MP + 5MP + 2MP,16MP Front Camera,16.94 cm (6.67 inch),2400 x 1080 Pixels,Qualcomm Snapdragon 732G,Octa Core,5020 mAh,"₹23,143",https://www.flipkart.com/redmi-note-10-pro-max...


# 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [23]:
# opening google maps
driver.get("https://www.google.co.in/maps")
time.sleep(3)

city = input('Enter City Name : ')                                         # Enter city to be searched
search = driver.find_element_by_id("searchboxinput")                       # locating search bar
search.clear()                                                             # clearing search bar
time.sleep(2)
search.send_keys(city)                                                     # entering values in search bar
button = driver.find_element_by_id("searchbox-searchbutton")               # locating search button
button.click()                                                             # clicking search button
time.sleep(3)

try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e))

Enter City Name : Bhubaneswar
URL Extracted:  https://www.google.co.in/maps/place/Bhubaneswar,+Odisha/@20.1910045,84.4359044,7z/data=!4m5!3m4!1s0x3a1909d2d5170aa5:0xfc580e2b68b33fa8!8m2!3d20.2960587!4d85.8245398
Latitude = 20.1910045, Longitude = 84.4359044


# 6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [24]:
driver.get('https://trak.in/')

In [25]:
button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(button)

In [26]:
fund_dict = {}
fund_dict['Date'] = []
fund_dict['Startup Name'] = []
fund_dict['Industry/Vertical'] = []
fund_dict['Sub-Vertical'] = []
fund_dict['Location'] = []
fund_dict['Investor'] = []
fund_dict['Investment Type'] = []
fund_dict['Amount(in USD)'] = []

In [27]:
for i in range(48,51):
    driver.find_element_by_xpath('//div[@id="tablepress-{}_wrapper"]/div/label/select/option[4]'.format(i)).click()

    # Date
    dt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in dt:
        fund_dict['Date'].append(d.text)

    # Startup Name
    sn = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in sn:
        fund_dict['Startup Name'].append(n.text)
    
    # Industry/Vertical
    ind = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind:
        fund_dict['Industry/Vertical'].append(n.text)
    
    # Sub-Vertical
    sv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        fund_dict['Sub-Vertical'].append(s.text)

    # Location
    loc = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in loc:
        fund_dict['Location'].append(l.text)
    
    # Investor
    inv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in inv:
        fund_dict['Investor'].append(n.text)
    
    # Investment Type
    invt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        fund_dict['Investment Type'].append(n.text)
        
    # Amount
    amt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amt:
        fund_dict['Amount(in USD)'].append(a.text)
    
fund_df = pd.DataFrame(fund_dict)
fund_df

,Date,Startup Name,Industry/Vertical,Sub-Vertical,Location,Investor,Investment Type,Amount(in USD)
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


In [30]:
fund_df.to_csv("Indian Startups_Q2_2020.csv")

# 7. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [37]:
driver.get('https://www.digit.in/')

# click on 'Best gaming laptops' link
driver.find_element_by_xpath('/html/body/div[3]/div/div[2]/div[2]/div[4]/ul/li[9]/a').click()

time.sleep(3)

# get laptop names
names=[]
names.clear()

names_wbe=driver.find_elements_by_xpath("//h3")

for i in names_wbe:
    names.append(i.text)
    
time.sleep(2)

# get specifications box weblements
spec_wbe=driver.find_elements_by_xpath("//div[@class='Spcs-details']")

specs=[]
specs.clear()

for wbe in spec_wbe:
    specs.append(wbe.text)

In [38]:
names

['ALIENWARE AREA 51M R2',
 'ALIENWARE M15 R3',
 'ASUS ROG STRIX SCAR 15',
 'ASUS ROG ZEPHYRUS G14',
 'LENOVO LEGION 5I',
 'ASUS ROG ZEPHYRUS DUO 15',
 'ACER ASPIRE 7 GAMING']

In [39]:
specs

['SPECIFICATION\nOS : Windows 10 Home\nDisplay : 17.3" (1920 x 1080)\nProcessor : 10th Generation Intel® Core™ i7-10700 | 2.90 GHz\nMemory : 1 TB SSD/16 GBGB DDR4\nWeight : 4.1\nDimension : 27.65 x 402.6 x 319.14\nGraphics Processor : Intel® UHD Graphics 630',
 'SPECIFICATION\nOS : Windows 10 Home\nDisplay : 15.6" (3840 x 2160)\nProcessor : 10th Generation Intel® Core™ i9-10980HK | NA\nMemory : 1 TB SSD/16 GBGB DDR4\nWeight : NA\nDimension : NA\nGraphics Processor : NA\nPrice : ₹341990',
 'SPECIFICATION\nOS : Windows 10 Home\nDisplay : 15.6" (1920 x 1080)\nProcessor : AMD Ryzen™ 9 5900HX | 3.3 GHz\nMemory : 1 TB SSD/16 GBGB DDR4\nWeight : 2.30\nDimension : 35.4 x 25.9 x 2.26\nGraphics Processor : NVIDIA® GeForce RTX™ 3070',
 'SPECIFICATION\nOS : Windows 10 Home\nDisplay : 14" (1920 x 1080)\nProcessor : AMD 3rd Generation Ryzen 9 | 3.3 GHz\nMemory : 1 TB SSD/16 GBGB DDR4\nWeight : 1.65\nDimension : 32.5 x 22.1 x 1.8\nGraphics Processor : NVIDIA GeForce RTX 2060',
 'SPECIFICATION\nOS : W

In [40]:
temp_specs=[]
for specs in specs:
    temp_specs.append(specs.split('\n'))
temp_specs[0]

['SPECIFICATION',
 'OS : Windows 10 Home',
 'Display : 17.3" (1920 x 1080)',
 'Processor : 10th Generation Intel® Core™ i7-10700 | 2.90 GHz',
 'Memory : 1 TB SSD/16 GBGB DDR4',
 'Weight : 4.1',
 'Dimension : 27.65 x 402.6 x 319.14',
 'Graphics Processor : Intel® UHD Graphics 630']

In [41]:
temp_specs

[['SPECIFICATION',
  'OS : Windows 10 Home',
  'Display : 17.3" (1920 x 1080)',
  'Processor : 10th Generation Intel® Core™ i7-10700 | 2.90 GHz',
  'Memory : 1 TB SSD/16 GBGB DDR4',
  'Weight : 4.1',
  'Dimension : 27.65 x 402.6 x 319.14',
  'Graphics Processor : Intel® UHD Graphics 630'],
 ['SPECIFICATION',
  'OS : Windows 10 Home',
  'Display : 15.6" (3840 x 2160)',
  'Processor : 10th Generation Intel® Core™ i9-10980HK | NA',
  'Memory : 1 TB SSD/16 GBGB DDR4',
  'Weight : NA',
  'Dimension : NA',
  'Graphics Processor : NA',
  'Price : ₹341990'],
 ['SPECIFICATION',
  'OS : Windows 10 Home',
  'Display : 15.6" (1920 x 1080)',
  'Processor : AMD Ryzen™ 9 5900HX | 3.3 GHz',
  'Memory : 1 TB SSD/16 GBGB DDR4',
  'Weight : 2.30',
  'Dimension : 35.4 x 25.9 x 2.26',
  'Graphics Processor : NVIDIA® GeForce RTX™ 3070'],
 ['SPECIFICATION',
  'OS : Windows 10 Home',
  'Display : 14" (1920 x 1080)',
  'Processor : AMD 3rd Generation Ryzen 9 | 3.3 GHz',
  'Memory : 1 TB SSD/16 GBGB DDR4',
  'W

In [42]:
temp_specs[0][3][0:3]

'Pro'

In [43]:
os=[]
dis=[]
processor=[]
memory=[]
weight=[]
dimension=[]
graphic=[]
price=[]

In [44]:
for j in temp_specs:
    if len(j)==8:
        for i in j:
            if i[0]=='O':
                os.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0:3]=='Dis':
                dis.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0:3]=='Pro':
                processor.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0]=='M':
                memory.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0]=='W':
                weight.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0:3]=='Dim':
                dimension.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0]=='G':
                graphic.append(i.split(':')[-1])
                time.sleep(2)
                price.append('--')
                continue
    elif len(j)==9:
        for i in j:
            if i[0]=='O':
                os.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0:3]=='Dis':
                dis.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0:3]=='Pro':
                processor.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0]=='M':
                memory.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0]=='W':
                weight.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0:3]=='Dim':
                dimension.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0]=='G':
                graphic.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0:3]=='Pri':
                price.append(i.split(':')[-1])
                time.sleep(2)
                continue              

In [45]:
df_digit=pd.DataFrame({})
df_digit['Model Name']=names
df_digit['Operating System']=os
df_digit['Display']=dis
df_digit['Processor']=processor
df_digit['Memory']=memory
df_digit['Weight']=weight
df_digit['Dimension']=dimension
df_digit['Graphics Processor']=graphic
df_digit['Price']=price
df_digit

,Model Name,Operating System,Display,Processor,Memory,Weight,Dimension,Graphics Processor,Price
0,ALIENWARE AREA 51M R2,Windows 10 Home,"17.3"" (1920 x 1080)",10th Generation Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,--
1,ALIENWARE M15 R3,Windows 10 Home,"15.6"" (3840 x 2160)",10th Generation Intel® Core™ i9-10980HK | NA,1 TB SSD/16 GBGB DDR4,NA,NA,NA,₹341990
2,ASUS ROG STRIX SCAR 15,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 9 5900HX | 3.3 GHz,1 TB SSD/16 GBGB DDR4,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070,--
3,ASUS ROG ZEPHYRUS G14,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Generation Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,--
4,LENOVO LEGION 5I,Windows 10 Pro,"15.6"" (1920 x 1080)",10th Generation Intel® Core™ i5-10300H | 2.50...,1 TB SSD/16 GBGB DDR4,2.3,363.06 x 259.61 x 23.57,NVIDIA® GeForce® GTX 1650 4GB,₹71990
5,ASUS ROG ZEPHYRUS DUO 15,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,--
6,ACER ASPIRE 7 GAMING,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 5-5500U hexa-core | NA,512 GB SSD/8 GBGB DDR4,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650,₹56990


In [46]:
df_digit.to_csv("Gaming laptops_digit.csv")

# 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”,“Industry”.

In [16]:
# opening www.forbes.com
driver.get('https://www.forbes.com/')

In [17]:
# link of world billionaires link.
billionaires =driver.find_element_by_xpath("//a[@class='section__link']").get_attribute('href')

In [18]:
# Activate required browser.
driver.get(billionaires)

In [19]:
#intialising lists
Age=[]
Rank=[]
Name=[]
Source=[]
NetWorth=[]
Industry=[]
Citizenship=[]

#scraping Rank
try:
    for i in driver.find_elements_by_xpath("//div[@class='rank']"):
        Rank.append(i.text)
except NoSuchElementException:
    Rank.append("_")

#scraping names
try:
    for i in driver.find_elements_by_xpath("//div[@class='personName']"):
        Name.append(i.text)
except NoSuchElementException:
    Name.append("_")

#scraping Networth
try:
    for i in driver.find_elements_by_xpath("//div[@class='netWorth']"):
        NetWorth.append(i.text)
except NoSuchElementException:
    NetWorth.append("_")
    
#scraping Age
try:
    for i in driver.find_elements_by_xpath("//div[@class='age']"):
        Age.append(i.text)
except NoSuchElementException:
    Age.append("_")
    
#scraping Citizenship
try:
    for i in driver.find_elements_by_xpath("//div[@class='countryOfCitizenship']"):
        Citizenship.append(i.text)
except NoSuchElementException:
    Citizenship.append("_")
    
#scraping Source
try:
    for i in driver.find_elements_by_xpath("//div[@class='source-column']"):
        Source.append(i.text)
except NoSuchElementException:
    Source.append("_")
    
#scraping Industry
try:
    for i in driver.find_elements_by_xpath("//div[@class='category']"):
        Industry.append(i.text)
except NoSuchElementException:
    Industry.append("_")

In [21]:
# convert all list into Dataframe
pd.set_option('display.max_rows',None)
Billionaires=pd.DataFrame({})
Billionaires['Age']=Age
Billionaires['Name']=Name
Billionaires['Source']=Source
Billionaires['Industry']=Industry
Billionaires['NetWorth']=NetWorth
Billionaires['Citizenship']=Citizenship
Billionaires

,Age,Name,Source,Industry,NetWorth,Citizenship
0,57,Jeff Bezos,Amazon,Technology,$177 B,United States
1,49,Elon Musk,"Tesla, SpaceX",Automotive,$151 B,United States
2,72,Bernard Arnault & family,LVMH,Fashion & Retail,$150 B,France
3,65,Bill Gates,Microsoft,Technology,$124 B,United States
4,36,Mark Zuckerberg,Facebook,Technology,$97 B,United States
5,90,Warren Buffett,Berkshire Hathaway,Finance & Investments,$96 B,United States
6,76,Larry Ellison,software,Technology,$93 B,United States
7,48,Larry Page,Google,Technology,$91.5 B,United States
8,47,Sergey Brin,Google,Technology,$89 B,United States
9,63,Mukesh Ambani,diversified,Diversified,$84.5 B,India


# 9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [23]:
# opening www.youtube.com
driver.get('https://www.youtube.com/')

In [24]:
# link of vedio.
vedio=driver .find_element_by_xpath("//a[@class='yt-simple-endpoint inline-block style-scope ytd-thumbnail']").get_attribute('href')
driver.get(vedio)
time.sleep(5)

In [25]:
# Scrolling the windows for more comments
for _ in range(500):
    driver.execute_script("window.scrollBy(0,1000)")
time.sleep(10)

In [26]:
#Scrapping the comments
Coments=[]
for i in driver.find_elements_by_id("content-text"):
    Coments.append(i.text)
time.sleep(5)
    
#Scrapping the comment timing
Time=[]
for i in driver.find_elements_by_id("header-author"):
    Time.append(i.text)
time.sleep(5)

In [27]:
# Made Dataframe of variables
pd.set_option('display.max_rows',None)
You_tube=pd.DataFrame({})
You_tube['Comments']=Coments
You_tube['Upvote/time']=Time
You_tube

,Comments,Upvote/time
0,"yeah he's Ryan ""freakin"" Reynolds, the guy wit...",Virtual Playlist\n1 week ago
1,despite being 54 years old .. salma hayek is g...,Abdul Rahim\n12 hours ago
2,"Wow! That is ""well written"" script.",Philip Marlowe\n3 days ago
3,Somehow this movie looks amazing,lololololololol ol\n2 weeks ago
4,"ever since deadpool ,Reynolds is pretty much h...",Takudzwa Makopa\n2 weeks ago
5,Salma is the hottest thing in Hollywood!,Robert Hadden\n2 weeks ago
6,"Wow,🔥 This movie looks amazing. Salma is the h...",Indian Viral Videos\n5 hours ago
7,You don't need guns when you carry 2 missiles ...,John Wick\n2 weeks ago
8,"Ryan - ""Where's your shirt?""\n\nEveryone else ...",Ali\n4 days ago
9,Those two cannons on her chest are just enough...,Porfyrius\n1 week ago (edited)


# 10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [8]:
def hostelinfo(place):
    driver.get('https://www.hostelworld.com/') #loading url
    driver.maximize_window() #maximizing window
    time.sleep(2)

    #Clicking on the search box
    driver.find_element_by_xpath('//input[@id="location-text-input-field"]').click()
    time.sleep(2)
    
    #Typing place provided
    driver.find_element_by_xpath('//input[@id="search-input-field"]').send_keys(place)

    #CLicking on the first predicted result
    WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="predicted-search-results"]/li[2]')))
    driver.find_element_by_xpath('//*[@id="predicted-search-results"]/li[2]').click()
    
    #Clicking on the search button
    driver.find_element_by_xpath('//button[@id="search-button"]').click()
    time.sleep(5)
    
    url=[]
    name=[]
    dist=[]
    rat=[]
    rev=[]
    orev=[]
    pprice=[]
    dprice=[]
    fac=[]
    propdesc=[]

    #Extracting info from first page
    
    #Extracting url
    for i in driver.find_elements_by_xpath('//h2[@class="title title-6"]/a'):
        url.append(i.get_attribute('href'))
        
    #Extracting distance from centre
    for i in driver.find_elements_by_xpath('//a[@class="show-on-map"]/span[1]'):
        dist.append(i.text)

    #Extracting prices
    for i in driver.find_elements_by_xpath('//div[@class="prices-col"]/a[1]'):
        #p
        if i.find_element_by_xpath('div[1]/p').text=='Privates From':
            pprice.append(i.find_element_by_xpath('div[1]/div').text)
        else:
            pprice.append('No Privates Available')
        #d
        if i.find_element_by_xpath('div[2]/p').text=='Dorms From':
            dprice.append(i.find_element_by_xpath('div[2]/div').text)
        else:
            dprice.append('No Dorms Available')
            
    #Extracring info from rest of the pages
    for page in driver.find_elements_by_xpath('//div[@class="pagination-item"]'):
        page.click()
        time.sleep(2)
        
        #Extracting from rest of the pages for each hostel
        for i in driver.find_elements_by_xpath('//h2[@class="title title-6"]/a'):
            url.append(i.get_attribute('href'))

        #Extracting distance from centre
        for i in driver.find_elements_by_xpath('//a[@class="show-on-map"]/span[1]'):
            dist.append(i.text)

        #Extracting prices
        for i in driver.find_elements_by_xpath('//div[@class="prices-col"]/a[1]'):
            #p
            if i.find_element_by_xpath('div[1]/p').text=='Privates From':
                pprice.append(i.find_element_by_xpath('div[1]/div').text)
            else:
                pprice.append('No Privates Available')
            #d
            if i.find_element_by_xpath('div[2]/p').text=='Dorms From':
                dprice.append(i.find_element_by_xpath('div[2]/div').text)
            else:
                dprice.append('No Dorms Available') 
                
     #Extracting info from each url
    for u in url:
        driver.get(u)

        #Extracting name
        name.append(driver.find_element_by_xpath('//div[@class="title-2"]').text)
        #Extracting rating
        try:
            rat.append(driver.find_element_by_xpath('//div[@class="score orange big"]').text)
        except NoSuchElementException:
            rat.append('No Rating')
        #Extracting review
        try:
            rev.append(driver.find_element_by_xpath('//div[@class="reviews"]').text)
        except NoSuchElementException:
            rev.append('0 Total Reviews')
        #Extracting overallrating
        orev.append(driver.find_element_by_xpath('//div[@class="keyword"]/span').text)
        #Extracting facilities
        s=''
        for j in driver.find_elements_by_xpath('//ul[@class="facility-sections"]/li[1]/ul/li'):
            s=s+j.text+'/'
        fac.append(s)
        #Extracting Property Description
        try:
            propdesc.append(driver.find_element_by_xpath('//div[@class="flex-80"]/div[2]/div/div[2]').text)
        except NoSuchElementException:
            propdesc.append(driver.find_element_by_xpath('//div[@class="flex-80"]/div/div/div[2]').text) 
            
    #Creating Dataframe
    data=pd.DataFrame()
    data['Name']=name
    data['Distance from Centre']=dist
    data['Ratings']=rat
    data['Review']=rev
    data['Overall Review']=orev
    data['Private Room Price']=pprice
    data['Dorm Price']=dprice
    data['Facilities']=fac
    data['Property Description']=propdesc
    return data
    

In [10]:
df=hostelinfo('London')
df

,Name,Distance from Centre,Ratings,Review,Overall Review,Private Room Price,Dorm Price,Facilities,Property Description
0,St Christopher's Village,Hostel - 1.8km from city centre,8.9,10814 Total Reviews,Fabulous,Rs3408,Rs1332,Linen Included/Free City Maps/Free WiFi/Free I...,COVID 19 Policy Update.\nIn response to Corona...
1,Generator London,Hostel - 3km from city centre,7.5,6729 Total Reviews,Very Good,Rs8572,Rs2169,Linen Included/Free City Maps/Free WiFi/Free I...,Generator London is a design hotel-hostel loca...
2,Safestay London Kensington Holland Park,Hostel - 5.9km from city centre,7.9,1070 Total Reviews,Very Good,Rs4181,Rs1022,Linen Included/Free WiFi/,Safestay Holland Park\n\nSafestay Holland Park...
3,"PubLove @ The Crown, Battersea",Hostel - 4.7km from city centre,7.9,204 Total Reviews,Very Good,Rs11898,Rs1239,Linen Included/Free City Maps/Free WiFi/,Do London like a local.\nStay just outside the...
4,Leman House,Hostel - 3.6km from city centre,No Rating,0 Total Reviews,No Rating,Rs5577,No Dorms Available,Free WiFi/Free Internet Access/,People come from different parts of the world ...
...,...,...,...,...,...,...,...,...,...
81,Park Hotel Essex,Hotel - 24.1km from city centre,No Rating,0 Total Reviews,No Rating,Rs3684,No Dorms Available,Free Breakfast/Free Parking/,This Hotel is the right choice for visitors wh...
82,Cranbrook Hotel,Hotel - 14.8km from city centre,No Rating,0 Total Reviews,No Rating,Rs3098,No Dorms Available,Free Breakfast/Free Parking/,We are located about twenty minutes by tube fr...
83,St. Athans,Bed and Breakfast - 2.9km from city centre,No Rating,0 Total Reviews,No Rating,Rs3950,No Dorms Available,Linen Included/Free City Maps/Towels Included/...,"The St Athans Hotel is a proudly simple, famil..."
84,Aron Guest House,Bed and Breakfast - 13.1km from city centre,No Rating,0 Total Reviews,No Rating,Rs6713,No Dorms Available,Free Parking/Towels Included/Free WiFi/,"Aron Guest House provides an excellent base, w..."
